In [1]:
import json
import pandas as pd
import os,sys

Load the JSON file and select only the messages from the JSON file to a pandas data frame

In [2]:
with open("data/result.json", "r" ,encoding='utf8') as file:
    data = json.load(file)
msgs = data['messages']
dataFrame = pd.DataFrame(msgs)

Filtering the DataFrame and keeping only the relevant columns for analyzing replies in a chat conversation.

In [4]:

messages = dataFrame.filter(items = ['id','from', 'reply_to_message_id' ,'text'])
messages.dropna(inplace = True)
replies = messages.loc[messages.text.str.len()> 2] 
dropList = ['how','when','which','what','same',"dm me",'good morning','guy','intested','good evening','goodafternoon','bro','awesome','thank you','thanks','thank','share','introduce','welcome','great','unfortunately','congratulations','congrats','wow' , 'please' , 'arun' , 'interested' , 'intereste' , 'ping']
replies = replies[ ~replies['text'].str.lower().str.contains('|'.join(dropList),na=False)]
#eliminate where they answering by a qustion 
replies = replies[~replies['text'].str.contains("?", regex=False)]


In [5]:
 replies

,id,from,reply_to_message_id,text
65,66,Dharma Teja,28.0,👍yes he will explain more clearly
254,280,Meer Abdul Kaleel K,277.0,So even we can test rest apis in postman
324,355,Arun Bansal,354.0,depends on your api functionalities
368,416,Mich,414.0,Did you got framework
369,417,Prasanta Sahoo,416.0,Nope
...,...,...,...,...
51674,65102,nenavath Mahender,64756.0,Hybrid page object patern
51717,66979,Surya,66978.0,I can guide on Security Testing
51732,66997,😊,66996.0,Yes I want
51739,67005,Chandra,66990.0,Yes


In [6]:
include_list =['tell','can','what','where','when','how','which','who','why','suggest','take' , 'please', 'explain' , 'java' , 'python' , 'fix' , 'sql', 'server', 'library']
remove_list =['pin','great','mrs','joined','dm','whatsapp','thanks','good morning','good evening', 'gud nite', 'good night', 'good afternoon','meant','hope','qualification']

questions = messages.loc[ ~messages['from'].isin(['Arun Bansal']) & (messages['text'].str.lower().str.contains('|'.join(include_list),na=False))  & (messages['text'].str.lower().str.len() < 101 )]
questions = questions[~questions['text'].str.lower().str.contains('|'.join(remove_list),na=False)]
questions

,id,from,reply_to_message_id,text
65,66,Dharma Teja,28.0,👍yes he will explain more clearly
254,280,Meer Abdul Kaleel K,277.0,So even we can test rest apis in postman
287,313,Nilesh@GAP,311.0,This is language code which we use in server
365,413,Kajal,409.0,If someone has framework please share.\nWill b...
444,494,Mahesh Konni,493.0,Do you know how to check for response time in ...
...,...,...,...,...
51662,65079,John,65066.0,Yes i can provide
51673,65101,Aruna kotha,65058.0,In which location nd how to contact
51717,66979,Surya,66978.0,I can guide on Security Testing
51740,67006,Chandra,67000.0,What about for 1year of experience


In [29]:
chat = pd.merge(questions, replies , left_on='id', right_on='reply_to_message_id')
chat

,id_x,from_x,reply_to_message_id_x,text_x,id_y,from_y,reply_to_message_id_y,text_y
0,494,Mahesh Konni,493.0,Do you know how to check for response time in ...,496,Arun Bansal,494.0,no never worked on swagger
1,891,Jaswender Nagi,890.0,How is ApI testing connected to Database ?,895,Arun Bansal,891.0,api gets the data or update the data from data...
2,977,Dave,939.0,Which world time zone,980,Arun Bansal,977.0,IST
3,1731,Pavan Kumar,1724.0,"thank you, can you pls explain more how to che...",1736,Arun Bansal,1731.0,This u need to ask to app dev / or api dev th...
4,1785,ÌÌÌÌÌ ÌÌÌÌÌ,1783.0,I have basic knowledge of SQL Queries. Any rea...,1786,Arun Bansal,1785.0,without db knowledge api knowledge is in compelte
...,...,...,...,...,...,...,...,...
204,61483,❤ Reddy,61482.0,Currently in which company he is working?,61484,Prasad Rupnar,61483.0,"Cyber tech solution, Pune"
205,62248,CoursesOnline,62246.0,Search free courses on udemy u can enroll free...,62249,sri 🌸,62248.0,Tqq
206,62443,Bi K,61778.0,How much it costs,62587,manoj,62443.0,25k
207,63657,S@gr,63656.0,Sry! It was my mistake bro,63658,Vinay Kumar,63657.0,np. Chill


remove the question with the specifique name of admin 

In [30]:
chat = chat[['text_x' , 'text_y']]
index = [i for i , row in chat.iterrows() if 'arun' in row['text_x']]

chat.drop(index=index , inplace = True)

<ipython-input-30-bda3fa82967d>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chat.drop(index=index , inplace = True)


now we wil se a sample of our cleaned chat :

In [9]:
sample = chat.sample(5)
for index , row in sample.iterrows():
    print(f"qst > {row['text_x']}")
    print(f"ans > {row['text_y']}")

qst > Can u give support
ans > Yes
qst > I provide online support . Please let me know if you need
ans > I need offline.
qst > Select employee_name from employee where employee_name like 'a%r'
ans > It's right
qst > Its difficult to take print out and scan from shops in this lockdown.
ans > Check for possibility
qst > For which role?
ans > Qa or I am open to go for product owner role too


With this chat, we can now seamlessly export it to a text file and organize it according to our preferences. I personally prefer to separate each question and answer with a comma, on separate lines, for the purpose of using it as a data feed for a chatbot utilizing the ChatterBot library.

In [31]:
"""with open('telegram_1.txt', 'a', encoding='utf8') as f:
    for index, row in chat.iterrows():
        f.write("Question: " + str(row['text_x']) + '\n')
        f.write("Answer: " + str(row['text_y']) + '\n')
"""

with open('telegram_1.txt', 'w', encoding='utf8') as f:
    for index, row in chat.iterrows():
        f.write(f" {row['text_x']} , {row['text_y']} ")


In [36]:
import nltk
with open("telegram_1.txt", 'r',  encoding='utf8') as chat : 
    text = chat.read()
    sentences = nltk.sent_tokenize(text)
for sentence in sentences :
    sentence.split(',')
    print(sentence + str(1))


 Do you know how to check for response time in swagger page , no never worked on swagger  How is ApI testing connected to Database ?1
, api gets the data or update the data from database and do some manipulation  Which world time zone , IST  thank you, can you pls explain more how to check the logs , This u need to ask to app dev  / or api dev that where do we save logs  I have basic knowledge of SQL Queries.1
Any reason for this ?1
, without db knowledge api knowledge is in compelte  S, please use Newman or supertest.1
, Let me know if u need further assistance 😊  Backend framework?1
Please name , Django
ExpressJS
Flask  There are some paid tools available.1
For that and you can also test using free vpn softwares , Install vpn and choose the country you will come to know that site is accessible or not in that country  How much time does it take to learn java for software testing , 2 months  How much time does it take to learn java for software testing , Why dont u try python .. for me

In [41]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# load the text data from a .txt file
data = pd.read_csv("telegram_1.txt", delimiter="\t", header=None, names=["text", "label"])

# preprocessing the text data
data["text"] = data["text"].str.lower()
data["text"] = data["text"].str.replace("[^a-zA-Z0-9]", " ")

# split the data into training and test sets
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

# vectorize the text data using CountVectorizer
vectorizer = CountVectorizer()
X_train = vectorizer.fit_transform(train_data["text"])
X_test = vectorizer.transform(test_data["text"])

train_data.fillna("", inplace=True)
test_data.fillna("", inplace=True)


# train a Multinomial Naive Bayes model
clf = MultinomialNB()
clf.fit(X_train, train_data["label"])

# make predictions on the test data
predictions = clf.predict(X_test)

# evaluate the model's performance
accuracy = accuracy_score(test_data["label"], predictions)
precision = precision_score(test_data["label"], predictions, average="weighted")
recall = recall_score(test_data["label"], predictions, average="weighted")
f1 = f1_score(test_data["label"], predictions, average="weighted")

print("Accuracy: ", accuracy)
print("Precision: ", precision)
print("Recall: ", recall)
print("F1-Score: ", f1)


Accuracy:  1.0
Precision:  1.0
Recall:  1.0
F1-Score:  1.0


<ipython-input-41-fabf9580dd0e>:12: FutureWarning: The default value of regex will change from True to False in a future version.
  data["text"] = data["text"].str.replace("[^a-zA-Z0-9]", " ")
<ipython-input-41-fabf9580dd0e>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data.fillna("", inplace=True)
<ipython-input-41-fabf9580dd0e>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data.fillna("", inplace=True)
